In [6]:
import os
import json
import pandas as pd
import requests
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variable
key = os.getenv("Hugging_Face_API_key")

In [3]:

# API URL for the Hugging Face model (change the model ID as needed)
API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-1B"
headers = {"Authorization": f"Bearer {key}"}

In [4]:
def query(payload):
    """Function to query the Hugging Face Inference API."""
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [15]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [7]:
# Template for quiz generation
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [8]:
# Evaluation template for quiz
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [18]:
def generate_quiz(text, number, subject, tone, response_json):
    """Function to generate and evaluate the quiz using the Hugging Face Inference API."""
    
    # Create payload for quiz generation
    quiz_payload = {
        "inputs": quiz_generation_prompt.format(text=text, number=number, subject=subject, tone=tone, response_json=response_json)
    }

    # Query the API for quiz generation
    quiz_response = query(quiz_payload)
     # Assuming quiz_response is a list, take the first entry
    quiz = quiz_response[0]["generated_text"] if isinstance(quiz_response, list) else quiz_response.get("generated_text", "")

    # Create payload for quiz evaluation
    evaluation_payload = {
        "inputs": quiz_evaluation_prompt.format(subject=subject, quiz=quiz)
    }

    # Query the API for quiz evaluation
    evaluation_response = query(evaluation_payload)
    # Assuming evaluation_response is a list, take the first entry
    review = evaluation_response[0]["generated_text"] if isinstance(evaluation_response, list) else evaluation_response.get("generated_text", "")

    return quiz, review

In [19]:
file_path=r"C:\Users\jnvpg\mcqgen2\data.txt"

In [20]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [21]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [22]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [23]:
# Example usage
text = TEXT
number = NUMBER
subject = SUBJECT
tone = TONE
response_json = json.dumps(RESPONSE_JSON)  # Example JSON response for guide

In [24]:
# Generate and evaluate the quiz
quiz, review = generate_quiz(text, number, subject, tone, response_json)

In [25]:
# Print the results
print("Generated Quiz:")
print(quiz)
print("\nReview:")
print(review)

Generated Quiz:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitiv